                                  **Housing Price Prediction ***
    
    Here i am using the Ameshousing Dataset(https://www.kaggle.com/prevek18/ames-housing-dataset) for my project in kaggle. i have done the Data Cleaning, Feature Engineering and applied the different learning algorithms to predict the housing price. 
    
    The Detailed Data Descriptions are available in https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data
    
    

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

First i have read the AmesHousing dataset using pandas and save it as a dataframe in the name of dataset.

In [ ]:
dataset = pd.read_csv("../input/ames-housing-dataset/AmesHousing.csv")

Next i have used the "dataset.shape" code to know the number of rows and columns in the dataset.
The result of the code shows that the dataset is having 2930 rows and 82 columns. That means the dataset is having the data of 2930 houses and each house(row) is having 82 features.

In [ ]:
dataset.shape

In [ ]:
dataset = pd.read_csv("../input/ames-housing-dataset/AmesHousing.csv")
dataset.head(n=5)

Then to know the nature of dataset i have printed the first five rows of dataset using head(5) comment. From the result we come to know that dataset is having both the numerical and object(non-numeric)type of features. And also we can note that the dataset is having missing values. The learning algorithm won't accept missing values(NaN). So, first we need to list out the numerical and object columns separtely and number of missing values in each column. Then We have to replace the missing values with '0' for numerical columns and with a suitable pharse for object type columns.

In [ ]:
dataset.describe(include=[np.number])

In [ ]:
dataset.describe(include=[np.object])

# **Data Preparation :**

    Before applying the learning lgorithm, we need to prepare the dataset as suitable to apply the learning algorithms. Data Prepration invoves Data Cleaning, Feature Engineering, Converting object type features to numerical in order to convert the whole dataset as numerical as the learning algorithms accept numerical data alone.
    
    
#     ** Data Cleaning**

          In the Data prepartion first we have to clean our dataset. Data cleaning means we have to remove the missing values, redundant rows and columns.And also we can remove the outliers in the dataset. 
          
          
  ** Replacing Missing values**
  
    First we are going to find the missing values replace with suitable values.

In [ ]:
num_missing = dataset.isnull().sum()

num_missing = num_missing[num_missing > 0]

num_missing

From the above list of missing value columns,i planned to replace missing values coulmn by column in the decreasing order of number of missing values.Note that "Pool QC" column is having highest(2917) number of missing values.so we are going to replace the missing values (NaN) in  "Pool QC" first. 

 For replacing the missing values, it is best practice to know more about the feature and find out why the values are missing, is there any specific condition behind the missing values. 
 
  To know more about the "Pool QC" feature go through the data describtion page (https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data).From describtion it is evidence that "Pool Qc" is the column which describe the swimming pool quality and their values(phrases).From that it is nown that 'NA' value is given for houses with no pool. Then when i tried to find out other columns related to swimming pool, we come to know about "Pool Area" column which have the area of swimming pool. 
  Now we are going to list out the unique values in "Pool Area". From the result we note that 'Pool Area' is having specific numeric value which represent the swimming pool area and also some houses are having '0' as value for 'Pool Area'. It can be intrepreted as the houses without swimming pool are having 'Pool Area' as '0'. Now we are going to count number of houses having "Pool Area' as '0'.
    

In [ ]:
dataset['Pool Area'].unique().tolist()

In [ ]:
dataset["Pool Area"].value_counts()

From the "Pool Area" value count result, we note that 2917 rows(houses) are having pool area as '0', which is the same number of houses which are having missing values(NaN) in "Pool Qc" column.So, let us cross check that wheather all the houses with missing values in Pool Qc are having Pool Area as '0'. 

In [ ]:
poolcol=[col for col in dataset.columns if "Pool" in col]

dataset[(dataset["Pool QC"].isnull()) & (dataset["Pool Area"]==0)][poolcol]

By listing out the rows which are having 'Pool Area' as '0' and 'Pool Qc' as null, we got 2917 rows.so it is understood that for all the houses without swimming pool the 'Pool QC' is missing. So in order to replace the missing value in Pool QC which is object type feature, we can use "No Pool" phrase by using fillna statement.

In [ ]:
dataset["Pool QC"].fillna("No Pool", inplace=True)

For verifying our replacement, let check the Pool QC value count. 

In [ ]:
dataset["Pool QC"].value_counts()

From the list of missing value count, the next column(feature) having highest(2824) missing value is 'Misc Feature'. From the data describtion, it is known that 'Misc Feature' coulmn is describing Miscellaneous feature which are not covered in other categories and it is not applicable('NA') for houses which are no other Miscellaneous feature. Then we can find out that 'Misc Val' is another column related to miscellaneous features. 'Misc Val' is descrbing the value of Miscellaneous feature.
In order to understand the relation between the missig values in 'Misc Feature' and 'Misc Val'column, First list out the value count for 'Misc Val'.

In [ ]:
dataset["Misc Val"].value_counts()

From the result, it is come know that 2827 rows(houses) are having 'Misc Val' as '0'. This number is much close to number of houses with missing values in 'Misc Feature'(2824). So, let us cross check that all the rows having missing values in "Misc Feature" are having '0' as value in 'Misc Val' column.

In [ ]:
Misccol=[col for col in dataset.columns if "Misc" in col]


dataset[(dataset["Misc Feature"].isnull()) & (dataset["Misc Val"]==0)][Misccol]

As for the above code we got 2824 rows in reslut, it is evidence that all the rows with missing values in 'Misc Feature' are having  value '0' in 'Misc Val'. we can also list out the rows which are having values in Misc Feature but '0' in Misc Val column. It can be intrepreted as the resultant three rows(houses) are exceptional to have value '0' in 'Misc Val' may be because of poor quality  Miscellaneous Feature even though it is having some  Miscellaneous feature.

In [ ]:
dataset[(~dataset["Misc Feature"].isnull()) & (dataset["Misc Val"]==0)][Misccol]

So it can be interpreted as all the 2824 (rows) houses which are having missing values in "Misc Feature' and '0 value in 'Misc Val' are not having any Misc features.Hence we can replace missing values with the phrase 'No feature'.

In [ ]:
dataset['Misc Feature'].fillna('No feature', inplace=True)

Let cross check ou replacement.

In [ ]:
dataset["Misc Feature"].value_counts()

The next highest missing value column is 'Alley' Column.As per the data description, Alley is the feature which dscribes the Type of alley access to property and there is two type of alley. Then the feature is not applicable(NA) if there is no alley for the house. finally there is no other column related to alley. First find out the value out of alley, in order to verify all the rows are having only the valid type of alley.

In [ ]:
dataset["Alley"].value_counts()

From the result we can confirm that all the rows are having one of the valid alley type or else the value is missing. so, we can replace the missing value(NaN) as 'No Alley'

In [ ]:
dataset['Alley'].fillna('No Alley', inplace=True)

let cross check our replacement

In [ ]:
dataset["Alley"].value_counts()

The next highest missing value column is 'Fence' Column.As per the data description, Fence is the feature which describes the Quality of Fence and there is four quality levels. Then the feature is not applicable(NA) if there is no fence for the house. finally there is no other column related to Fence. First find out the value out of Fence, in order to verify all the rows are having only the valid Quality level.

In [ ]:
dataset["Fence"].value_counts()

From the result we can confirm that all the rows are having one of the valid Fence Quality or else the value is missing. so, we can replace the missing value(NaN) as 'No Fence'

In [ ]:
dataset['Fence'].fillna('No Fence', inplace=True)

In [ ]:
dataset["Fence"].value_counts()

Similare to Alley and Fence for 'Fireplace Qu' feature also replace missing values with 'No Fireplace' phrase.  

In [ ]:
dataset['Fireplace Qu'].fillna('No Fireplace', inplace=True)

Next for 'Lot Frontage' feature which represent the length of street connected to the house in feet.As it is  a numerical feature and there is no other column related to Lot Frontage, it is best to replace the missing values with '0'

In [ ]:
dataset['Lot Frontage'].fillna(0, inplace=True)

In the next level, columns related to garage are having highest number missing values.From the Data describtion, let us listout the columns related to gargage, 
   1.Garage Cond, 
   2.Garage Qual,
   3.Garage Finish, 
   4.Garage Yr Blt, 
   5.Garage Type, 
   6.Garage Cars, and 
   7.Garage Area
   
   Among these the Garage Cars and Garage Area are numeric type features which mention about the size of Gargage in terms of Cars capacity and Square feet respectively. 
   Garage Yr Blt column is a numerical column represents the year in which the garage is build. 
   
 From the list of columns having missing values, we come to know that, Garage Cond, Garage Qual, Garage Finish, Garage Yr Blt columns have 159 missing values.Garage Type has 157 missing values. That means for two rows there is value for Garage Type alone while the values are missing for all other garage columns. 

 
 Let look at those two rows
 
 


In [ ]:

garage_columns = [col for col in dataset.columns if col.startswith("Garage")]
dataset[~(dataset['Garage Type'].isnull()) & 
            (dataset['Garage Qual'].isnull())][garage_columns]



From the result , we note that there is no meaningful data available related to garage in these rows other than the type. So, we can fill the coumns as 'no Garage' for object type features and fill with '0' fo numerical type features.

 Garage Cars and Garage Area have one missing value. 
 
 For the 157 rows(houses) without garage they filled the garage Area and Garage Cars columns with '0', but unfortunatley they may miss to fill '0' in one row. That's what now we got 1 missing value in Garage Area and Garage Cars. In the above result, the row number 2236 is having missing values in Garage Cars and Garage Area for which we already decided to fill missing values with '0'.
 
  Anyway just verify our interpretation. 
   

In [ ]:
dataset["Garage Cars"].value_counts()

In [ ]:
dataset["Garage Area"].value_counts()

From the above two code,it is verified that 157 rows with out garage are filled with '0' in garage Area and Garage Cars column. Now will fill the missing values. First fill the missing value in Garage Cars and Area in row 2236 with 0.



In [ ]:
dataset['Garage Cars'].fillna(0, inplace=True)
dataset['Garage Area'].fillna(0, inplace=True)

In [ ]:
dataset["Garage Area"].value_counts()

Then for the two rows (2236 and 1356) fill the Garage Type column with 'No garage' which are having value for Garage Type alone.  

In [ ]:
dataset.loc[~(dataset['Garage Type'].isnull()) & 
            (dataset['Garage Qual'].isnull()),"Garage Type"] = "No Garage"

In [ ]:
dataset[~(dataset['Garage Type'].isnull()) & 
            (dataset['Garage Qual'].isnull())][garage_columns]


Now we can fill the missing values in all the 159 rows.

In [ ]:
for col in ['Garage Type', 'Garage Finish', 'Garage Qual', 'Garage Cond']:
    dataset[col].fillna('No Garage', inplace=True)
    
dataset['Garage Yr Blt'].fillna(0, inplace=True)

Before going to next features with missing values, just verify our replacement by list out the missing values count once again.

In [ ]:
num_missing = dataset.isnull().sum()

num_missing = num_missing[num_missing > 0]

num_missing

From the above list, it is confirmed that our replacements of missing values are sucessfull untill the Garage columns. Now We can look at the columns related to Basement with missing values.


Columns related to Basements are Bsmt Exposure, BsmtFin Type 2, BsmtFin Type 1, Bsmt Qual, Bsmt Cond, Bsmt Half Bath, Bsmt Full Bath, Total Bsmt SF, Bsmt Unf SF, BsmtFin SF 2, and BsmtFin SF 1

From the above list. we got the count of missing values as Bsmt Cond, Bsmt Qual , BsmtFin Type 1 are having 80 missing values. Bsmt Exposure is having 83 missing values and Bsmtfin Type 2 is having 81 missing values.

  So, we can intepret as there is no basement at all in the 80 houses(rows). as the 80 rows are having missing values in Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1 and BsmtFin Type 2. 
   
   Then for three more houses there are missing values in Bsmt Exposure Column alone. Let us look at those three rows.
   
  


In [ ]:
bsmt_columns = [col for col in dataset.columns if "Bsmt" in col]
dataset[~(dataset['Bsmt Cond'].isnull()) & 
    dataset['Bsmt Exposure'].isnull()][bsmt_columns]

From the above result, it is cleared that these three rows are having basement but there is no data related to basement exposure.Hence we can interpret as these houses are not having basement exposure and fill these missing values with 'No' as per the data description 'No' means the house is not having basement exposure.

In [ ]:
dataset.loc[~(dataset['Bsmt Cond'].isnull()) & 
    dataset['Bsmt Exposure'].isnull(), "Bsmt Exposure"] = "No"

 Similarly other than 80 rows, 1 more row is having missing values in BsmtFin Type 2 alone. let look at the row which is having missing value in BsmtFin type 2 but having value in Bsmt Qual, Bsmt Cond and also in BsmtFin Type 1.


In [ ]:
dataset[~(dataset['Bsmt Cond'].isnull()) & 
        (dataset['BsmtFin Type 2'].isnull())][bsmt_columns]

From the above result, we got to know that, the row number 444 is having valid data in BsmtFin SF 2 which represents the surface area of BsmtFin Type 2. That means , the houses is having BsmtFin Type 2 but unforunately the rating of BsmtFin Type 2 data is missing in the column BsmtFin Type 2. So, we can fill the column with the least ranked Type

In [ ]:
dataset.loc[~(dataset['Bsmt Cond'].isnull()) & 
            (dataset['BsmtFin Type 2'].isnull()), "BsmtFin Type 2"] = "Unf"

The numerical Basement features such BsmtFin SF1, BsmtFin SF 2, Bsmt unf SF, total Bsmt SF are having one missing value each and Bsmt Full Bath ,Bsmt Half Bath are having two missing values.that means for the rows(houses) without basement, or with basement but without basement Fin type 1 or Basement Fin Type 2 , they filled the numerical features with '0'.But mistakenly for one row values are missing in BsmtFin SF1,BsmtFin SF2, Bsmt unf SF and Total Bsmt SF. let look at that row.

In [ ]:
dataset[(dataset["BsmtFin SF 1"].isnull())][bsmt_columns]

In [ ]:
dataset[dataset['Bsmt Half Bath'].isnull()][bsmt_columns]

We can replacethe missing missing values in these numerical columns with '0'

In [ ]:
for col in ["Bsmt Half Bath", "Bsmt Full Bath", "Total Bsmt SF", 
            "Bsmt Unf SF", "BsmtFin SF 2", "BsmtFin SF 1"]:
    dataset[col].fillna(0, inplace=True)

Now we can replace the missing values in all object type columns for 80 rows which are not having basement with 'No Basement'

In [ ]:
for col in ["Bsmt Exposure", "BsmtFin Type 2", 
            "BsmtFin Type 1", "Bsmt Qual", "Bsmt Cond"]:
    dataset[col].fillna("No Basement", inplace=True)

Just verify wheather our replacemnts are successful. 

In [ ]:
num_missing = dataset.isnull().sum()

num_missing = num_missing[num_missing > 0]

num_missing

 Now look at the columns Mas Vnr Type and Mas Vnr Area.They represents the type of Masonry veneer and its area in square feet.There is no other coulmn related to Masonry veneer and as both the columns are having 23 missing values commonly. We can replace the missing values of Mas Vnr Area with '0' as it is numerical type feature. And from data describtion, it is known that if the house is not having Masonry veneer, the Mas Vnr Type is given 'None'.Hence we can replace the missing values of Mas Vnr Type with 'None'.

In [ ]:
dataset['Mas Vnr Type'].fillna('none', inplace=True)
dataset['Mas Vnr Area'].fillna(0, inplace=True)

In [ ]:
dataset['Mas Vnr Type'].isnull().sum() # check if null values have been replaced with 'none'

At last we have only one missing value in Electrical column. Electrical column is represent the type of electrical system in the house. We known that all the houses are mandatory having electrical facility. But mistakenly the type of electrical system is missing in one house. So it is best practice to find the type of electrical system found in most of the house and replace the missing value with te most occured value. The most occured value can be found by using mode() method.

In [ ]:
most=dataset['Electrical'].mode()[0]
most

In [ ]:
dataset['Electrical'].fillna(most, inplace=True)

Atlast we removed all the missing values in our dataset and replaced with suitable value.

In [ ]:
num_missing = dataset.isnull().sum()

num_missing = num_missing[num_missing > 0]

num_missing

**Removing Redundant Rows and Columns**

In the Next step of data cleaning we have to find whether the dataset is having any redendant rows and remove them if any.By using the duplicated() function we can the duplicated rows.

In [ ]:
dupl_set=dataset[dataset.duplicated()] # we're just taking duplicated rows and storing it in dupl_set. 
dupl_set

The Above results shows zero rows, that means the dataset is not having any duplicate rows. Then we can find the duplicated columns in the dataset and remove them. To find the duplicate columns, columns.duplicated() can be used.

In [ ]:
dataset.columns[dataset.columns.duplicated()]

The above result shows that, the dataset is not having any duplicte columns.

**Removing Outliners**

Next we can find the outliners in the dataset and remove them as the presence of outliner would affect the efficiency of the algorithm . By the Data set provider information, the dataset is having five main outliners,based on Gir Liv Area. Hence first we can visvalise the outliners based on Gir Liv Area vs SalesPrice.

In [ ]:
from matplotlib import pyplot as plt # pyplot is a package to display graph
plt.scatter(x=dataset['Gr Liv Area'], y=dataset['SalePrice'], # plt.scattter means it'll display each point as a dot
            color="green", edgecolors="#000000", linewidths=0.5);
plt.xlabel("Gr Liv Area"); plt.ylabel("SalePrice");

From the result, we can find that there are only 5 rows(houses) are Gir Liv Area above 4000 and their salesprice is also having deviation from others.Hence we can remove these rows from the dataset for the better result in learning algorithm.

In [ ]:
dataset = dataset[dataset["Gr Liv Area"] < 4000]
dataset.reset_index(drop=True, inplace=True)
dataset.shape


Now we just removed the outliner rows mentioned by the data provider, which have high deviation in Gir Liv Area from the remaining rows. behind this outliner of dataset based on several factor can be found by sklearn packages. but it is best to apply them on training set after the training and test set separation. So, lets apply the sklearn outlinear detection package after training and test set spitting. Now we are going to next step, Feature Engineering.

# **Feature Engineering**

**Removing uninflunced Columns 

After Data Cleaning we are going to analysis our features of dataset. Our dataset is having 82 Columns(features).In the feature engineering,first we are going to remove the columns which are having very little or no influence on our target variable(SalesPrice). Among the 82 columns, the 'Order' and 'PID' columns are used to represent the serialnumber and property id of the houses respectively. As they are used only for identification and having no contibution on Salesprice of the houses, we can remove them.

In [ ]:
dataset.drop(['Order', 'PID'], axis=1, inplace=True)

 We are going to find what are all features are havinng high corelation or high influence on SalesPrice and give them higher weightage on learing algorithm. And also we can find out the highly corelated features and remove on of them as they are nearly reprsenting similar data and having almost equal influence on salesprice. First we would find the corelation between features and salesprice by calling the cor function on dataset.

In [ ]:
corr_matrix = dataset.corr()
corr_matrix

The above resultant matrix represents the corelation value of each column of dataset with each other.Fromthis corelaton matrix we can filter out the highly corelated pairs. We are going to consider the corelation above 0.7 alone for feature engineering.

In [ ]:
high_cor_bol=(corr_matrix[corr_matrix.columns]>0.7)&(corr_matrix[corr_matrix.columns]!=1)
high_cor_row = high_cor_bol.any(axis=1)
high_cor_row = high_cor_row[high_cor_row == True]
rowNames=list(high_cor_row.index)
for row in rowNames:
    cols = list(high_cor_bol[row][high_cor_bol[row] == True].index)

high_cor_featurepairs = list()
for row in rowNames:
    cols = list(high_cor_bol[row][high_cor_bol[row] == True].index)
    for col in cols:
        high_cor_featurepairs.append((row, col,corr_matrix[row][col]))
for i in range(len(high_cor_featurepairs)):
    print(high_cor_featurepairs[i])

The above result shows the highly corelated column pairs. From this we come to know that Salesprice is highly corelated with OverallQual and Gir Liv Area. Hence we are going to add their polynomial square and cubes of the columns as new added columns. And also we are going to add their polynomial product as one more column in order to give more weigthage to these feature. 

In [ ]:
for f in ["Overall Qual", "Gr Liv Area"]: # common name we are giving these two columns is f
    dataset[f + "_p2"] = dataset[f] ** 2 # ** means power. + is used for concentation to create a new column name
    dataset[f + "_p3"] = dataset[f] ** 3



In [ ]:
dataset["OverallQual_GrLivArea"] = dataset["Overall Qual"] * dataset["Gr Liv Area"]


From the highly corelated pairs, we also come to know that Total Bsmt SF', '1st Flr SF' . So we can remove one among them which are less corelation with salesprice. Similarly Garage Area has high correlation with Garage Cars and Gr Liv Area'has high corelation with  'TotRms AbvGrd.  So,we would remove one of them.

In [ ]:
dataset.drop(['Garage Area','1st Flr SF', 'TotRms AbvGrd'], axis=1, inplace=True)

In [ ]:
dataset.columns

# **Converting Object type features to numerical**

   Among object type features some are ordinal features and some are categorial features. 
   
 **Convert Object type Ordinal features to numerical ordinal features.**
   
   The ordinal features represents ranking in string form, first we convert these ordinal features to numerical by replacing the string form rank with numerical ranks.
   
   First list out the object type ordinal features from data describtion
   
   1. Bsmt Cond
2.Bsmt Qual
3. Bsmt Exposure
4. BsmtFin Type 1
5. BsmtFin Type 2
6. Exter Qual
7. Exter Cond
8. Heating QC
9. Kitchen Qual
10,. Central Air 
11. Functional - 
12. Fireplace Qu
13. Garage Finish 
14. Garage Qual 
15. Garage Cond
16. Pool QC
17. Land Slope 
18. Fence 

We would assign the numerical ranks for all these 18 features using map() fuction.
   

In [ ]:
mp = {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'No Basement':0}
dataset['Bsmt Qual'] = dataset['Bsmt Qual'].map(mp)
dataset['Bsmt Cond'] = dataset['Bsmt Cond'].map(mp)
dataset['Bsmt Exposure'] = dataset['Bsmt Exposure'].map(
    {'Gd':4,'Av':3,'Mn':2,'No':1,'No Basement':0})

In [ ]:
mp = {'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1,'No Basement':0}
dataset['BsmtFin Type 1'] = dataset['BsmtFin Type 1'].map(mp)
dataset['BsmtFin Type 2'] = dataset['BsmtFin Type 2'].map(mp)


In [ ]:
mp = {'Ex':4,'Gd':3,'TA':2,'Fa':1,'Po':0}
dataset['Exter Qual'] = dataset['Exter Qual'].map(mp)
dataset['Exter Cond'] = dataset['Exter Cond'].map(mp)
dataset['Heating QC'] = dataset['Heating QC'].map(mp)
dataset['Kitchen Qual'] = dataset['Kitchen Qual'].map(mp)

In [ ]:
dataset['Central Air'] = dataset['Central Air'].map({'Y':1,'N':0})
dataset['Functional'] = dataset['Functional'].map(
    {'Typ':7,'Min1':6,'Min2':5,'Mod':4,'Maj1':3,
     'Maj2':2,'Sev':1,'Sal':0})
dataset['Fireplace Qu'] = dataset['Fireplace Qu'].map(
    {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'No Fireplace':0})
dataset['Garage Finish'] = dataset['Garage Finish'].map(
    {'Fin':3,'RFn':2,'Unf':1,'No Garage':0})
dataset['Garage Qual'] = dataset['Garage Qual'].map(
    {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'No Garage':0})
dataset['Garage Cond'] = dataset['Garage Cond'].map(
    {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'No Garage':0})
dataset['Pool QC'] = dataset['Pool QC'].map(
    {'Ex':4,'Gd':3,'TA':2,'Fa':1,'No Pool':0})
dataset['Land Slope'] = dataset['Land Slope'].map(
    {'Sev': 2, 'Mod': 1, 'Gtl': 0})
dataset['Fence'] = dataset['Fence'].map(
    {'GdPrv':4,'MnPrv':3,'GdWo':2,'MnWw':1,'No Fence':0})

Now we sucessfully converted all the object type ordinal columns into numerical. But still dataset is having some object type columns which are representing categorial features.

In [ ]:
dataset.describe(include=[np.object]) 

From the above result, it is confirm that our dataset is still having 25 object type features. Now we are going to convert them into numerical by using dummies() method. The dummies() method create separate columns for each categories in a  particular categorial object type column and assign 1 if the row is having a specific category or else 0. 

In [ ]:
dataset = pd.get_dummies(dataset)

In [ ]:
dataset.describe(include=[np.number]) 

# Feature Scaling 

Before applying the learing algorithm, it is always adviced to apply the feature scaling to improve the efficiency. As our dataset is having wide range of values and large numeric values, without scaling the algorithm will take more time for numerical calculation. So, we can scale our feature and use the relative values in algorithm we reduce the time taken for mathematical calculation and improves efficiency.

we are going to scale our feature using sklearn package.Here we are not scaling our target column(SalesPrice) in order to get salesprice prediction in actual range instead of scaled range. 

So, first we split our dataset in to two as feature and target. 

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

feature=dataset.drop('SalePrice', axis=1);
target=dataset['SalePrice'];

In [ ]:
feature.loc[:] = scaler.fit_transform(feature.loc[:])

# Training and Test dataset Splitting 

After feature scaler, we have to split our dataset into training set and test test. Training test is used to train our learning algorithms and test set would be used to test the efficiency our algorithms.

In [ ]:
from sklearn.model_selection import train_test_split 
feature_train, feature_test, target_train, target_test = train_test_split(feature,target, 
    test_size=0.25, random_state=2)

In [ ]:
feature_train.shape

In [ ]:
feature_test.shape

From  the above results, it is clear that training set is having 2193 records and test set is having 732 records.

# outliers removal in using sklearn package

In [ ]:
from sklearn.ensemble import IsolationForest 
from sklearn.metrics import mean_absolute_error

In [ ]:
iso = IsolationForest(contamination=0.1);
yhat = iso.fit(feature_train).predict(feature_train) ; 
mask=np.where(yhat==1)


The above code run the outliners detection algorithm and assign '0' as for outliners and '1' for outliners in the vector yhat. And 'mask' variable have the indices of inliners. 

In [ ]:
feature_train= feature_train.iloc[mask]
target_train=target_train.iloc[mask]


The above code locate the inliner records alone and store them in training set. So now our training dataset will have only the inliners. That means from our training dataset, we succesully removed the outliners.Now our dataset is having only 1973 records.

In [ ]:
target_train.shape


# Applying suitable learning models 

Now our dataset is completely ready for learning algoithms usgae. As Our problem statement is about predicting the housing price(continous variable) based on its features,it is naturally a linear algorithm model. So, first we can try the best linear algorithms. And then we would apply the advanced models like neural networks, Support Vector Regression , XGBoost

We are going to use the GridSearchCv model selection to select the best parameter combinations for the models.


# Elastic Model (Linear model)

We are going to use the Elastic model, as the GrideSearchCv itself would try the ridge model, Lasso model separately ,the combination of ridge and lasso model by tuning the l1_ratio parameter approrimately and gives us the best parameter combination for the best model among the three linear model(ridge, lasso and elastic). if L1_ratio  is 0 then its Ridge. If its 1 then its Lasso. If its in between its Elastic.

In [ ]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import ElasticNet # importing elastic model 

parameter_space = {
    "alpha": [1, 10, 100, 280, 500], # alpha parameter
    "l1_ratio": [0,0.5,0.8, 1], # L1_ratio is the value for l1 penalty
    "fit_intercept": [True, False],
}

clf = GridSearchCV(ElasticNet(random_state=3), parameter_space, 
                   n_jobs=4, cv=3, scoring="neg_mean_absolute_error")

clf.fit(feature_train, target_train)
print("Best parameters:")
print(clf.best_params_)


The best parameter of Elasticnet model gives us the l1_ratio as '1', which means the lasso modelis best among ridge,elastic and lasso model.Now we can use the best parameter of elastic model to train our dataset.Then we test the model using the test dataset.

In [ ]:
elasticNet_model = ElasticNet(random_state=3, **clf.best_params_)
elasticNet_model.fit(feature_train, target_train);
target_pred = elasticNet_model.predict(feature_test)
elasticNet_mae = mean_absolute_error(target_test, target_pred)
print("Elastic Net MAE =", elasticNet_mae)

The above result shows that, the elastic model gives us the efficiency in terms of MAE(Mean absolute Error) 13831.86 in the test dataset.

**Random Forest (Cloassification based model)**

Now we are going to try for the next model, RandomForest. Though the Random Forest is naturally as classifier algorithm, most of the it is suitable for linear model to predict the values which are mostly falls in the same range of values. So, we try for randomforest algorithm too.

In [ ]:

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

parameter_space = \
    {
        "n_estimators": [10, 100, 300, 600],
        "criterion": ["mse", "mae"],
        "max_depth": [7, 50, 254],
        "min_samples_split": [2, 5],
        "min_samples_leaf": [1, 5],
        "max_features": [19, 100, feature_train.shape[1]],
        "bootstrap": [True, False],
    }

clf = RandomizedSearchCV(RandomForestRegressor(random_state=3), 
                         parameter_space, cv=3, n_jobs=4,
                         scoring="neg_mean_absolute_error", 
                         n_iter=10, random_state=3)

clf.fit(feature_train, target_train)
print("Best parameters:")
print(clf.best_params_)

In [ ]:
rf_model = RandomForestRegressor(**clf.best_params_)
rf_model.fit(feature_train, target_train);
target_pred = rf_model.predict(feature_test)
rf_mae = mean_absolute_error(target_test, target_pred)
print("Random Forest MAE =", rf_mae)


We got the efficiency of 14733.01 in terms of MAE for random forest algorithm.

**Neural Networks**

Next we try for neural networks algorithm.

In [ ]:
from sklearn.neural_network import MLPRegressor

parameter_space = \
    {
        "hidden_layer_sizes": [(7,)*3, (19,), (100,), (154,)],
        "activation": ["identity", "logistic", "tanh", "relu"],
        "solver": ["lbfgs"],
        "alpha": [1, 10, 100],
      
    }

clf = GridSearchCV(MLPRegressor(random_state=3), parameter_space,
                   cv=3, scoring="neg_mean_absolute_error", n_jobs=4)

clf.fit(feature_train, target_train)
print("Best parameters:")
print(clf.best_params_)

In [ ]:
nn_model = MLPRegressor(**clf.best_params_,max_iter=300)
nn_model.fit(feature_train, target_train);
target_pred = nn_model.predict(feature_test)
nn_mae = mean_absolute_error(target_test, target_pred)
print("Neural Network MAE =", nn_mae)

We got the efficiency of 15574.66 in terms of MAE for neural networks algorithm.

**Support Vector Regression (SVR) **

Next we try for Support Vector Regression (SVR), which is based on support vector machine. The SVR improves the efficiency by parallel computation by using vector concept.	


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR

parameter_space = \
    {
        "kernel": ["poly", "linear", "rbf", "sigmoid"],
        "degree": [3, 5],
        "coef0": [0, 3, 7],
        "gamma":[1e-3, 1e-1, 1/feature_train.shape[1]],
        "C": [1, 10, 100],
    }

clf = GridSearchCV(SVR(), parameter_space, cv=3, n_jobs=4,
                   scoring="neg_mean_absolute_error")

clf.fit(feature_train, target_train)
print("Best parameters:")
print(clf.best_params_)

In [ ]:
svr_model = SVR(**clf.best_params_)
svr_model.fit(feature_train, target_train);
target_pred = svr_model.predict(feature_test)
svr_mae = mean_absolute_error(target_test, target_pred)
print("Support Vector Regression MAE =", svr_mae)

We got the efficiency of 12267.42 in terms of MAE for Support Vector Regression (SVR)

**Xgboost regression**

Finally we try for the advanced model xgboost regression which try for the combination of different model to provide us the best efficiency.


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb

from xgboost import XGBRegressor
#xgb_regressor = xgb.XGBRegressor()
parameter_space = \
    {
        "max_depth": [4, 5, 6],
        "learning_rate": [0.005, 0.009, 0.01],
        "n_estimators": [700, 1000, 2500],
        "booster": ["gbtree",],
        "gamma": [7, 25, 100],
        "subsample": [0.3, 0.6],
        "colsample_bytree": [0.5, 0.7],
        "colsample_bylevel": [0.5, 0.7,],
        "reg_alpha": [1, 10, 33],
        "reg_lambda": [1, 3, 10],
    }
clf = RandomizedSearchCV(XGBRegressor(random_state=3), 
                         parameter_space, cv=3, n_jobs=4,
                         scoring="neg_mean_absolute_error", 
                         random_state=3, n_iter=10)

clf.fit(feature_train, target_train)
print("Best parameters:")
print(clf.best_params_)


In [ ]:
xgb_model = XGBRegressor(**clf.best_params_)
xgb_model.fit(feature_train, target_train);
target_pred = xgb_model.predict(feature_test)
xgb_mae = mean_absolute_error(target_test, target_pred)
print("XGBoost MAE =", xgb_mae)

We got the efficiency of 12851.13 in terms of MAE for xgboost regression​ model. Now lets list out the efficiency of all the model and pick out the best one.


**    Model  and                      Efficiency (MAE)**



       Support Vector Regression       12267.42
       
       XGboost Regression              12851.13

       Elastic                         13542.49
       (Lasso)       
       
       Random Forest                   14733.01
       
       Neural Networks                 15574.66
       
       
   
       
       

From the above table of model and effciency, it is cleared that we got the good result in Elastic model as it is best suitable for our problem statement(Linear algorithm).Although our result got improved with the advanced models Support vector Regression and XGboost Regression. 

But the efficiency of neural networks and random forest are slightly lowered than the Elastic model. May be because of nature of the dataset .

In [ ]:
submission = pd.DataFrame({'Id': feature_test.index, 'SalePrice': target_pred})
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)
